In [10]:
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

In [11]:
img_transforms = transforms.Compose(
    [
        transforms.Resize(64),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
    ]
)

In [12]:
train_data_path = "./data/Training"
test_data_path = "./data/Testing"
val_data_path = "./data/Validation"

In [13]:
train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=img_transforms)
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=img_transforms)
val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform=img_transforms)

In [14]:
batch_size = 64
train_data_loader = DataLoader(train_data,batch_size=batch_size)
test_data_loader = DataLoader(train_data,batch_size=batch_size)
val_data_loader = DataLoader(train_data,batch_size=batch_size)

In [16]:
class CNNNet(nn.Module):
    def __init__(self):
        super(CNNNet,self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,64,kernel_size=11,stride=4,padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2),
            
            nn.Conv2d(64,192,kernel_size=5,padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2),
            
            nn.Conv2d(192,384,kernel_size=3,padding=1),
            nn.ReLU(),
            
            nn.Conv2d(384,256,kernel_size=3,padding=1),
            nn.ReLU(),
            
            nn.Conv2d(256,256,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3,stride=2)
            
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6,6))